In [1]:
import csv

import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt

RANDOM_SEED = 42

In [22]:
dataset = 'data.csv'
#model_save_path = 'model/keypoint_classifier/keypoint_classifier.hdf5'
#tflite_save_path = 'model/keypoint_classifier/keypoint_classifier.tflite'
model_save_path = 'new_datahand_classifier.hdf5'
tflite_save_path = 'new_datahand_classifier.tflite'

In [23]:
NUM_CLASSES = 24

In [24]:
arr = pd.read_csv(dataset, header=None, delimiter=";")
# check the shape
input_shape = arr.shape
print(arr.iloc[0])
print(arr.iloc[1])
print(f"The input shape of your array is: {input_shape}")

0    0,0,0,0,-3.519914,0.050068,3.621712,-6.490975,...
Name: 0, dtype: object
0    0,0,0,0,-3.519914,0.050068,3.621712,-6.498184,...
Name: 1, dtype: object
The input shape of your array is: (1215, 1)


In [27]:
X_dataset = np.loadtxt(dataset, delimiter=',', dtype='float32', usecols=list(range(1, 365)))
y_dataset = np.loadtxt(dataset, delimiter=',', dtype='int32', usecols=(0))
X_train, X_test, y_train, y_test = train_test_split(X_dataset, y_dataset, train_size=0.75, random_state=RANDOM_SEED)

In [28]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Reshape((182, 2, 1), input_shape=(364, )),
    tf.keras.layers.Conv2D(32, kernel_size=(3, 1), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 1), activation='relu', padding='same'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 1)),
    tf.keras.layers.Flatten(),
    #tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    #tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')
])

In [29]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 182, 2, 1)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 182, 2, 32)        128       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 91, 2, 32)         0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 91, 2, 64)         6208      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 45, 2, 64)         0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 5760)             

In [30]:
# Model checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [31]:
# Model compilation
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [32]:
history = model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[cp_callback, es_callback]
)

Epoch 1/1000
8/8 [==============================] - ETA: 0s - loss: 25.6591 - accuracy: 0.1405
Epoch 1: saving model to new_datahand_classifier.hdf5
8/8 [==============================] - 2s 61ms/step - loss: 25.6591 - accuracy: 0.1405 - val_loss: 13.5247 - val_accuracy: 0.2434
Epoch 2/1000
7/8 [=========================>....] - ETA: 0s - loss: 7.2775 - accuracy: 0.3114

C:\gesture_recognition_by_image\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: saving model to new_datahand_classifier.hdf5
8/8 [==============================] - 0s 30ms/step - loss: 7.2318 - accuracy: 0.3117 - val_loss: 4.1865 - val_accuracy: 0.4112
Epoch 3/1000
7/8 [=========================>....] - ETA: 0s - loss: 2.8375 - accuracy: 0.4944
Epoch 3: saving model to new_datahand_classifier.hdf5
8/8 [==============================] - 0s 31ms/step - loss: 2.8525 - accuracy: 0.4951 - val_loss: 2.4416 - val_accuracy: 0.5954
Epoch 4/1000
7/8 [=========================>....] - ETA: 0s - loss: 1.4488 - accuracy: 0.6540
Epoch 4: saving model to new_datahand_classifier.hdf5
8/8 [==============================] - 0s 31ms/step - loss: 1.4447 - accuracy: 0.6564 - val_loss: 1.5867 - val_accuracy: 0.7237
Epoch 5/1000
7/8 [=========================>....] - ETA: 0s - loss: 0.8753 - accuracy: 0.7857
Epoch 5: saving model to new_datahand_classifier.hdf5
8/8 [==============================] - 0s 31ms/step - loss: 0.8775 - accuracy: 0.7838 - val_loss: 1.4262 - val_accura

In [33]:
# Model evaluation
val_loss, val_acc = model.evaluate(X_test, y_test, batch_size=128)

3/3 [==============================] - 0s 6ms/step - loss: 0.6271 - accuracy: 0.9539
